# Lego Multiple Linear Regression

## Obtaining Data

The data has been provided to us as two database files, lego.db and Walmart.db, and a csv, economy_data.csv. We will pull all the data from these files, combine them where possible, and save the combined datasets to csv.

In [1]:
import sqlite3
import pandas as pd

First, we need to get the info on all the tables from the database.

In [2]:
def print_table_data(database_name):
    con = sqlite3.connect(database_name)
    cur = con.cursor()
    cur.execute("""SELECT name
                   FROM sqlite_master
                   WHERE type='table'
                   ORDER BY name;
                """)
    tables = [t[0] for t in cur.fetchall()]
    for table in tables:
        print(f'Table: {table}\nInfo:')
        cur.execute(f"""SELECT *
                        FROM {table};
                     """)
        df = pd.DataFrame(cur.fetchall())
        df.columns = [c[0] for c in cur.description]
        print(df.info())

In [3]:
print_table_data('data/lego.db')

Table: product_details
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 4 columns):
prod_id           744 non-null int64
prod_desc         744 non-null object
prod_long_desc    744 non-null object
theme_name        744 non-null object
dtypes: int64(1), object(3)
memory usage: 23.4+ KB
None
Table: product_info
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 4 columns):
prod_id        744 non-null int64
ages           744 non-null object
piece_count    744 non-null int64
set_name       744 non-null object
dtypes: int64(2), object(2)
memory usage: 23.4+ KB
None
Table: product_pricing
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 3 columns):
prod_id       10870 non-null int64
country       10870 non-null object
list_price    10870 non-null object
dtypes: int64(1), object(2)
memory usage: 254.9+ KB
None
Table: product_reviews
Info:
<class

All the tables use the same 'prod_id' value as their key, so combining is an easy join statement. After printing out some of the details of our new table, we save it to a csv so it can be easily read into a pandas dataframe later.

In [8]:
con = sqlite3.connect('data/lego.db')
cur = con.cursor()
cur.execute("""SELECT *
               FROM product_info
               JOIN product_details
               USING(prod_id)
               JOIN product_pricing
               USING(prod_id)
               JOIN product_reviews
               USING(prod_id);
            """)
lego_df = pd.DataFrame(cur.fetchall())
lego_df.columns = [c[0] for c in cur.description]
lego_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
prod_desc            10870 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
country              10870 non-null object
list_price           10870 non-null object
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    10870 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(4), int64(2), object(8)
memory usage: 1.2+ MB


In [9]:
lego_df

,prod_id,ages,piece_count,set_name,prod_desc,prod_long_desc,theme_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,6-12,277,Bird Island Egg Heist,blah,blahblah,Angry Birds™,US,29.99,2.0,4.0,Average,4.5,4.0
1,75822,6-12,168,Piggy Plane Attack,blah,blahblah,Angry Birds™,US,19.99,2.0,4.0,Easy,5.0,4.0
2,75821,6-12,74,Piggy Car Escape,blah,blahblah,Angry Birds™,US,12.99,11.0,4.3,Easy,4.3,4.1
3,21030,12+,1032,United States Capitol Building,blah,blahblah,Architecture,US,99.99,23.0,3.6,Average,4.6,4.3
4,21035,12+,744,Solomon R. Guggenheim Museum®,blah,blahblah,Architecture,US,79.99,14.0,3.2,Challenging,4.6,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10865,70903,7-14,254,The Riddler™ Riddle Racer,blah,blahblah,THE LEGO® BATMAN MOVIE,PT,42.6878,18.0,4.2,Easy,4.2,4.4
10866,70910,7-14,204,Scarecrow™ Special Delivery,blah,blahblah,THE LEGO® BATMAN MOVIE,PT,30.4878,8.0,4.1,Average,4.4,4.1
10867,70901,7-14,201,Mr. Freeze™ Ice Attack,blah,blahblah,THE LEGO® BATMAN MOVIE,PT,30.4878,19.0,4.1,Easy,4.6,4.4
10868,70913,6-12,141,Scarecrow™ Fearful Face-off,blah,blahblah,THE LEGO® BATMAN MOVIE,PT,18.2878,7.0,4.0,Easy,4.1,4.4


In [10]:
lego_df.to_csv('data/lego.csv')

Next, we do the same thing with the Walmart.db data. Again, the two tables use the same 'Store' value as an id, so they can be easily joined.

In [11]:
print_table_data('data/Walmart.db')

Table: sales
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452192 entries, 0 to 452191
Data columns (total 5 columns):
Store           452192 non-null int64
Dept            452192 non-null int64
Date            452192 non-null object
Weekly_Sales    452192 non-null float64
IsHoliday       452192 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 17.2+ MB
None
Table: store_details
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
Store    45 non-null int64
Type     45 non-null object
Size     45 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.2+ KB
None


In [12]:
con = sqlite3.connect('data/Walmart.db')
cur = con.cursor()
cur.execute("""SELECT *
               FROM sales
               JOIN store_details
               USING(Store);
            """)
walmart_df = pd.DataFrame(cur.fetchall())
walmart_df.columns = [c[0] for c in cur.description]
walmart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452192 entries, 0 to 452191
Data columns (total 7 columns):
Store           452192 non-null int64
Dept            452192 non-null int64
Date            452192 non-null object
Weekly_Sales    452192 non-null float64
IsHoliday       452192 non-null object
Type            452192 non-null object
Size            452192 non-null int64
dtypes: float64(1), int64(3), object(3)
memory usage: 24.1+ MB


In [13]:
walmart_df

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315
2,1,1,2010-02-19,41595.55,False,A,151315
3,1,1,2010-02-26,19403.54,False,A,151315
4,1,1,2010-03-05,21827.90,False,A,151315
...,...,...,...,...,...,...,...
452187,45,98,2012-09-28,508.37,False,B,118221
452188,45,98,2012-10-05,628.10,False,B,118221
452189,45,98,2012-10-12,1061.02,False,B,118221
452190,45,98,2012-10-19,760.01,False,B,118221


Finally, there is an economy_data.csv file. It has unique 'Store' and 'Date' values, similar to our walmart_df. We can merge the two dataframes and save the result as walmart.csv

In [14]:
economy_df = pd.read_csv('data/economy_data.csv')
economy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
Store           8190 non-null int64
Date            8190 non-null object
Temperature     8190 non-null float64
Fuel_Price      8190 non-null float64
MarkDown1       4032 non-null float64
MarkDown2       2921 non-null float64
MarkDown3       3613 non-null float64
MarkDown4       3464 non-null float64
MarkDown5       4050 non-null float64
CPI             7605 non-null float64
Unemployment    7605 non-null float64
IsHoliday       8190 non-null bool
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [15]:
economy_df

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False


In [16]:
walmart_combined_df = pd.merge(walmart_df, economy_df,  how='left', on=['Store','Date'])
walmart_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452192 entries, 0 to 452191
Data columns (total 17 columns):
Store           452192 non-null int64
Dept            452192 non-null int64
Date            452192 non-null object
Weekly_Sales    452192 non-null float64
IsHoliday_x     452192 non-null object
Type            452192 non-null object
Size            452192 non-null int64
Temperature     452192 non-null float64
Fuel_Price      452192 non-null float64
MarkDown1       161614 non-null float64
MarkDown2       120077 non-null float64
MarkDown3       147431 non-null float64
MarkDown4       145762 non-null float64
MarkDown5       162365 non-null float64
CPI             452192 non-null float64
Unemployment    452192 non-null float64
IsHoliday_y     452192 non-null bool
dtypes: bool(1), float64(10), int64(3), object(3)
memory usage: 59.1+ MB


In [17]:
walmart_combined_df

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452187,45,98,2012-09-28,508.37,False,B,118221,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684,False
452188,45,98,2012-10-05,628.10,False,B,118221,64.89,3.985,5046.74,NaN,18.82,2253.43,2340.01,192.170412,8.667,False
452189,45,98,2012-10-12,1061.02,False,B,118221,54.47,4.000,1956.28,NaN,7.89,599.32,3990.54,192.327265,8.667,False
452190,45,98,2012-10-19,760.01,False,B,118221,56.47,3.969,2004.02,NaN,3.18,437.73,1537.49,192.330854,8.667,False


In [18]:
walmart_combined_df.to_csv('data/walmart.csv')

## Scrubbing and Cleaning Data

## Exploring Data

## Modeling Data